In [6]:
%matplotlib qt


In [7]:
# ======================================================
# IMPORTS
# ======================================================
import cv2
import os
import numpy as np
from PIL import Image

# ======================================================
# CONFIGURATION
# ======================================================
INPUT_FOLDER = "input_images"
OUTPUT_FOLDER = "scans"

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# ======================================================
# UTILITY FUNCTIONS
# ======================================================

def save_with_increment(filename, image):
    base, ext = os.path.splitext(filename)
    counter = 0
    new_filename = filename

    while os.path.exists(new_filename):
        counter += 1
        new_filename = f"{base}({counter}){ext}"

    cv2.imwrite(new_filename, image)
    return new_filename


def get_unique_pdf_path(folder, base_name):
    if not base_name.lower().endswith(".pdf"):
        base_name += ".pdf"

    name, ext = os.path.splitext(base_name)
    counter = 0

    while True:
        candidate = f"{name}{ext}" if counter == 0 else f"{name}({counter}){ext}"
        full_path = os.path.join(folder, candidate)
        if not os.path.exists(full_path):
            return full_path
        counter += 1


def load_images_from_folder(folder):
    supported = (".jpg", ".jpeg", ".png", ".bmp")
    images = []

    if not os.path.exists(folder):
        print(f"[ERROR] Folder '{folder}' does not exist.")
        return images

    for file in sorted(os.listdir(folder)):
        if file.lower().endswith(supported):
            img = cv2.imread(os.path.join(folder, file))
            if img is not None:
                images.append((file, img))
    return images


# ======================================================
# MANUAL CROPPING (OPENCV — EXACTLY LIKE YOUR IMAGE)
# ======================================================

def manual_crop_opencv(image, window_name="Select ROI (ENTER=OK, ESC=Skip)"):
    """
    Allows precise cropping even for large images by
    scaling display but cropping original image.
    """

    h, w = image.shape[:2]

    # --- Determine scaling factor ---
    screen_w, screen_h = 1200, 800  # safe display size
    scale = min(screen_w / w, screen_h / h, 1.0)

    display_img = cv2.resize(
        image, (int(w * scale), int(h * scale)),
        interpolation=cv2.INTER_AREA
    )

    # --- Create resizable window ---
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.resizeWindow(window_name, display_img.shape[1], display_img.shape[0])

    # --- Select ROI on DISPLAY image ---
    roi = cv2.selectROI(window_name, display_img, showCrosshair=True, fromCenter=False)
    cv2.destroyWindow(window_name)

    x, y, rw, rh = roi
    if rw == 0 or rh == 0:
        return image  # skipped

    # --- Map ROI back to ORIGINAL image ---
    x = int(x / scale)
    y = int(y / scale)
    rw = int(rw / scale)
    rh = int(rh / scale)

    return image[y:y+rh, x:x+rw]

# ======================================================
# POST-CROP PROCESSING MODES
# ======================================================

def process_output(image, mode):
    """
    1 -> Normal (Color)
    2 -> Grayscale
    3 -> Binary (Scanner style)
    """
    if mode == "1":
        return image

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    if mode == "2":
        return gray

    if mode == "3":
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        binary = cv2.adaptiveThreshold(
            blur, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY,
            15, 5
        )
        return binary

    return image


# ======================================================
# PDF FUNCTION
# ======================================================

def save_images_to_pdf(image_paths, pdf_path):
    images = [Image.open(p).convert("RGB") for p in image_paths]
    images[0].save(pdf_path, save_all=True, append_images=images[1:])
    print(f"[PDF CREATED] {pdf_path}")


# ======================================================
# MAIN PIPELINE
# ======================================================

def main():
    user_pdf_name = input("Enter PDF name (without .pdf): ").strip()
    pdf_path = get_unique_pdf_path(OUTPUT_FOLDER, user_pdf_name)

    scanned_image_paths = []
    images = load_images_from_folder(INPUT_FOLDER)

    if not images:
        print("[INFO] No images found.")
        return

    print(f"[INFO] Found {len(images)} images.\n")

    for filename, image in images:
        print(f"[PROCESSING] {filename}")

        # 1️⃣ USER CROPS RAW IMAGE (OPENCV ROI)
        cropped = manual_crop_opencv(image)

        # 2️⃣ USER CHOOSES OUTPUT MODE
        print("Choose output mode:")
        print("1 → Normal (Color)")
        print("2 → Grayscale")
        print("3 → Binary (Scanner style)")
        mode = input("Enter choice (1/2/3): ").strip()

        final_output = process_output(cropped, mode)

        # 3️⃣ SAVE IMAGE
        saved_path = save_with_increment(
            os.path.join(OUTPUT_FOLDER, "scanned.png"),
            final_output
        )

        print(f"[SAVED] {saved_path}")

        # 4️⃣ ADD TO PDF?
        choice = input("Add this image to PDF? (y/n): ").strip().lower()
        if choice == "y":
            scanned_image_paths.append(saved_path)
            print("[INFO] Added to PDF.\n")
        else:
            print("[INFO] Skipped.\n")

    # 5️⃣ CREATE PDF
    if scanned_image_paths:
        save_images_to_pdf(scanned_image_paths, pdf_path)
        print("[SUCCESS] PDF generated successfully.")
    else:
        print("[INFO] No images selected for PDF.")


# ======================================================
# RUN
# ======================================================
main()

Enter PDF name (without .pdf):  first


[INFO] Found 5 images.

[PROCESSING] 1.jpeg
Choose output mode:
1 → Normal (Color)
2 → Grayscale
3 → Binary (Scanner style)


Enter choice (1/2/3):  2


[SAVED] scans\scanned(53).png


Add this image to PDF? (y/n):  y


...

[INFO] Added to PDF.

[PROCESSING] bill.jpg
Choose output mode:
1 → Normal (Color)
2 → Grayscale
3 → Binary (Scanner style)


Enter choice (1/2/3):  2


[SAVED] scans\scanned(57).png


Add this image to PDF? (y/n):  n


[INFO] Skipped.

[PDF CREATED] scans\first.pdf
[SUCCESS] PDF generated successfully.
